In [6]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

# RAIN-GT

file_path = "/content/drive/MyDrive/Colab Notebooks/CPSC 480-580: Computer Vision/Final/data/GT-RAIN_test"
output_path_haze = "/content/drive/MyDrive/Colab Notebooks/CPSC 480-580: Computer Vision/Final/data/GT-RAIN_test"

dir_list = os.listdir(file_path)

print("RAIN-GT")

sum_psnr_original = 0
sum_psnr_bilateral = 0
sum_psnr_guided = 0
sum_ssim_original = 0
sum_ssim_bilateral = 0
sum_ssim_guided = 0

for dir_name in dir_list:
    if dir_name.startswith('.'):
        continue # skip .DS_Store

    total_psnr_original = 0
    total_psnr_bilateral = 0
    total_psnr_guided = 0
    total_ssim_original = 0
    total_ssim_bilateral = 0
    total_ssim_guided = 0
    i = 0

    gt_name = [name for name in os.listdir(os.path.join(file_path, dir_name)) if name.endswith("C-000.png")]
    if not gt_name:
        continue  # Skip if no ground truth image is found in the directory

    gt_name = gt_name[0]
    gt_full_path = os.path.join(file_path, dir_name, gt_name)

    name_list = [name for name in os.listdir(os.path.join(file_path, dir_name)) if not name.endswith("C-000.png")]

    for name in name_list:
        if name.startswith('.'):
            continue # skip .DS_Store

        haze_full_path = os.path.join(output_path_haze, dir_name, name)

        if os.path.isfile(gt_full_path) and os.path.isfile(haze_full_path):
            gt = cv2.imread(gt_full_path, flags=1)
            img = cv2.imread(haze_full_path, flags=1)
            imgGuide = cv2.imread(haze_full_path, flags=1)
            imgBiFilter = cv2.bilateralFilter(img, d=2, sigmaColor=75, sigmaSpace=75)
            imgGuidedFilter = cv2.ximgproc.guidedFilter(imgGuide, img, 10, 2, -1)

            # compute psnr and ssim
            total_psnr_original += psnr(img, gt)
            total_psnr_bilateral += psnr(imgBiFilter, gt)
            total_psnr_guided += psnr(imgGuidedFilter, gt)

            total_ssim_original += ssim(img, gt, channel_axis = 2)
            total_ssim_bilateral += ssim(imgBiFilter, gt, channel_axis = 2)
            total_ssim_guided += ssim(imgGuidedFilter, gt, channel_axis = 2)

            i += 1

    # Calculate average PSNR and SSIM
    average_psnr_original = total_psnr_original / i
    average_psnr_bilateral = total_psnr_bilateral / i
    average_psnr_guided = total_psnr_guided / i

    average_ssim_original = total_ssim_original / i
    average_ssim_bilateral = total_ssim_bilateral / i
    average_ssim_guided = total_ssim_guided / i

    sum_psnr_original += average_psnr_original
    sum_psnr_bilateral += average_psnr_bilateral
    sum_psnr_guided += average_psnr_guided
    sum_ssim_original += average_ssim_original
    sum_ssim_bilateral += average_ssim_bilateral
    sum_ssim_guided += average_ssim_guided

    print(dir_name)

    print(f"PSNR (Original): {average_psnr_original}")
    print(f"PSNR (Bilateral): {average_psnr_bilateral}")
    print(f"PSNR (Guided Filter): {average_psnr_guided}")

    print(f"SSIM (Original): {average_ssim_original}")
    print(f"SSIM (Bilateral): {average_ssim_bilateral}")
    print(f"SSIM (Guided Filter): {average_ssim_guided}")

    print()

print("Average of PSNR (Original):", sum_psnr_original / 7)
print("Average of PSNR (Bilateral):", sum_psnr_bilateral / 7)
print("Average of PSNR (Guided):", sum_psnr_guided / 7)
print("Average of SSIM (Original):", sum_ssim_original / 7)
print("Average of SSIM (Bilateral):", sum_ssim_bilateral / 7)
print("Average of SSIM (Guided):", sum_ssim_guided / 7)


RAIN-GT
Oinari_1-1
PSNR (Original): 16.340532645216896
PSNR (Bilateral): 16.36727685151684
PSNR (Guided Filter): 16.341003022069923
SSIM (Original): 0.5092521535101904
SSIM (Bilateral): 0.49424388886560433
SSIM (Guided Filter): 0.5091487418745729

Oinari_0-0
PSNR (Original): 19.964709744220674
PSNR (Bilateral): 20.04253351441849
PSNR (Guided Filter): 19.966613081314883
SSIM (Original): 0.5276283600354871
SSIM (Bilateral): 0.5600090253738206
SSIM (Guided Filter): 0.528469670887516

Winter_Garden_0-1
PSNR (Original): 16.995620993097525
PSNR (Bilateral): 17.026422628143862
PSNR (Guided Filter): 16.99678096483599
SSIM (Original): 0.48243660752116263
SSIM (Bilateral): 0.47288343337879307
SSIM (Guided Filter): 0.4823944458648548

M1135_0-0
PSNR (Original): 20.05757665451009
PSNR (Bilateral): 20.03795838007286
PSNR (Guided Filter): 20.056179682305263
SSIM (Original): 0.6018772676344035
SSIM (Bilateral): 0.5975344732624965
SSIM (Guided Filter): 0.6016148746955885

Table_Rock_0-0
PSNR (Original

In [16]:
import cv2
import matplotlib.pyplot as plt
import os
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim


# A2I2

output_path_groundtruth = "/content/drive/MyDrive/Colab Notebooks/CPSC 480-580: Computer Vision/Final/data/UAV/groundtruth-rec"
output_path_haze = "/content/drive/MyDrive/Colab Notebooks/CPSC 480-580: Computer Vision/Final/data/UAV/haze-rec"

gt_list = os.listdir(output_path_groundtruth)
haze_list = os.listdir(output_path_haze)

total_psnr_original = 0
total_psnr_bilateral = 0
total_psnr_guided = 0

total_ssim_original = 0
total_ssim_bilateral = 0
total_ssim_guided = 0

i = 0

for name in gt_list:
    gt_full_path = os.path.join(output_path_groundtruth, name)
    haze_full_path = os.path.join(output_path_haze, name)

    gt = cv2.imread(gt_full_path, flags=1)
    img = cv2.imread(haze_full_path, flags=1)
    imgGuide = cv2.imread(haze_full_path, flags=1)
    imgBiFilter = cv2.bilateralFilter(img, d=2, sigmaColor=75, sigmaSpace=75)
    imgGuidedFilter = cv2.ximgproc.guidedFilter(imgGuide, img, 10, 2, -1)

    # compute psnr and ssim
    total_psnr_original += psnr(gt, img)
    total_psnr_bilateral += psnr(gt, imgBiFilter)
    total_psnr_guided += psnr(gt, imgGuidedFilter)

    total_ssim_original += ssim(gt, img, channel_axis = 2)
    total_ssim_bilateral += ssim(gt, imgBiFilter, channel_axis = 2)
    total_ssim_guided += ssim(gt, imgGuidedFilter, channel_axis = 2)

    i += 1


print("A2I2")

print("psnr original:", total_psnr_original / i)
print("psnr bilateral filter:", total_psnr_bilateral / i)
print("psnr guided filter:", total_psnr_guided / i)

print("ssim original:", total_ssim_original / i )
print("ssim bilateral filter:", total_ssim_bilateral / i )
print("ssim guided filter:", total_ssim_guided / i)


A2I2
psnr original: 15.733396137187395
psnr bilateral filter: 15.745613979948526
psnr guided filter: 15.736405991364029
ssim original: 0.65036853045525
ssim bilateral filter: 0.6556542425911674
ssim guided filter: 0.6528101679966223


In [15]:
import cv2
import matplotlib.pyplot as plt
import os
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim


# RESIDE-IN

output_path_groundtruth = "/content/drive/MyDrive/Colab Notebooks/CPSC 480-580: Computer Vision/Final/data/RESIDE-IN/GT"
output_path_haze = "/content/drive/MyDrive/Colab Notebooks/CPSC 480-580: Computer Vision/Final/data/RESIDE-IN/hazy"

gt_list = os.listdir(output_path_groundtruth)
haze_list = os.listdir(output_path_haze)

total_psnr_original = 0
total_psnr_bilateral = 0
total_psnr_guided = 0

total_ssim_original = 0
total_ssim_bilateral = 0
total_ssim_guided = 0

i = 0

for name in gt_list:
    gt_full_path = os.path.join(output_path_groundtruth, name)
    haze_full_path = os.path.join(output_path_haze, name)

    gt = cv2.imread(gt_full_path, flags=1)
    img = cv2.imread(haze_full_path, flags=1)
    imgGuide = cv2.imread(haze_full_path, flags=1)
    imgBiFilter = cv2.bilateralFilter(img, d=2, sigmaColor=75, sigmaSpace=75)
    imgGuidedFilter = cv2.ximgproc.guidedFilter(imgGuide, img, 10, 2, -1)

    # compute psnr and ssim
    total_psnr_original += psnr(gt, img)
    total_psnr_bilateral += psnr(gt, imgBiFilter)
    total_psnr_guided += psnr(gt, imgGuidedFilter)

    total_ssim_original += ssim(gt, img, channel_axis = 2)
    total_ssim_bilateral += ssim(gt, imgBiFilter, channel_axis = 2)
    total_ssim_guided += ssim(gt, imgGuidedFilter, channel_axis = 2)

    i += 1


print("RESIDE-IN")

print("psnr original:", total_psnr_original / i)
print("psnr bilateral filter:", total_psnr_bilateral / i)
print("psnr guided filter:", total_psnr_guided / i)

print("ssim original:", total_ssim_original / i )
print("ssim bilateral filter:", total_ssim_bilateral / i )
print("ssim guided filter:", total_ssim_guided / i)


RESIDE-IN
psnr original: 11.974825957068784
psnr bilateral filter: 11.960147944826117
psnr guided filter: 11.973171533116096
ssim original: 0.693425705104212
ssim bilateral filter: 0.6783541020804912
ssim guided filter: 0.6895523087086518


In [14]:
import cv2
import matplotlib.pyplot as plt

from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim


# RESIDE-OUT

output_path_groundtruth = "/content/drive/MyDrive/Colab Notebooks/CPSC 480-580: Computer Vision/Final/data/RESIDE-OUT/GT"
output_path_haze = "/content/drive/MyDrive/Colab Notebooks/CPSC 480-580: Computer Vision/Final/data/RESIDE-OUT/hazy"

gt_list = os.listdir(output_path_groundtruth)
haze_list = os.listdir(output_path_haze)

total_psnr_original = 0
total_psnr_bilateral = 0
total_psnr_guided = 0

total_ssim_original = 0
total_ssim_bilateral = 0
total_ssim_guided = 0

i = 0

for name in gt_list:
    gt_full_path = os.path.join(output_path_groundtruth, name)
    haze_full_path = os.path.join(output_path_haze, name)

    gt = cv2.imread(gt_full_path, flags=1)
    img = cv2.imread(haze_full_path, flags=1)
    imgGuide = cv2.imread(haze_full_path, flags=1)
    imgBiFilter = cv2.bilateralFilter(img, d=2, sigmaColor=75, sigmaSpace=75)
    imgGuidedFilter = cv2.ximgproc.guidedFilter(imgGuide, img, 10, 2, -1)

    # compute psnr and ssim
    total_psnr_original += psnr(gt, img)
    total_psnr_bilateral += psnr(gt, imgBiFilter)
    total_psnr_guided += psnr(gt, imgGuidedFilter)

    total_ssim_original += ssim(gt, img, channel_axis = 2)
    total_ssim_bilateral += ssim(gt, imgBiFilter, channel_axis = 2)
    total_ssim_guided += ssim(gt, imgGuidedFilter, channel_axis = 2)

    i += 1


print("RESIDE-OUT")

print("psnr original:", total_psnr_original / i)
print("psnr bilateral filter:", total_psnr_bilateral / i)
print("psnr guided filter:", total_psnr_guided / i)

print("ssim original:", total_ssim_original / i )
print("ssim bilateral filter:", total_ssim_bilateral / i )
print("ssim guided filter:", total_ssim_guided / i)


RESIDE-OUT
psnr original: 15.936805760455856
psnr bilateral filter: 15.884471304539778
psnr guided filter: 15.935326328455766
ssim original: 0.8054612651071359
ssim bilateral filter: 0.7786118642924489
ssim guided filter: 0.8050519044643977
